In [1]:
import geopandas as gpd

import pandas as pd

from zipfile import ZipFile

import xml.etree.ElementTree as ET

import re

import os

In [ ]:
# filter by intGapOrigin = 1 (native) (and Origin = Native for double check) and intGapPres = 1 (present) (and Presence = Known/extant for double check)

In [ ]:
for i in range(0, 5, 1):
    # loading the temp.zip and creating a zip object
    with ZipFile(
        f"/media/muskrat/T7 Shield/eco_data/v3/native/GAP/scraped/{i}.zip", "r"
    ) as zObject:

        # Extracting all the members of the zip
        # into a specific location.
        zObject.extractall(
            path=f"/media/muskrat/T7 Shield/eco_data/v3/native/GAP/unzipped/{i}"
        )

In [2]:
file_list = []
for i in range(4, 5, 1):
    #  list the files inside directory {i}
    for file_path in os.listdir(
        f"/media/muskrat/T7 Shield/eco_data/v3/native/GAP/unzipped/{i}"
    ):
        # print(file_path)
        # store each file path before extension in a list

        file_split = file_path.split(".")
        file_list.append(file_split[0])


# remove duplicates from file_list and preserve order
file_list = list(dict.fromkeys(file_list))

In [6]:
meta_df = pd.DataFrame()
name_df = pd.DataFrame()
geo_df = pd.DataFrame()
# for i in range(4, 5, 1):
i = 4
print(i)
tree = ET.parse(
    f"/media/muskrat/T7 Shield/eco_data/v3/native/GAP/unzipped/{i}/{file_list[0]}.xml"
)
root = tree.getroot()

name = ""
for title in root.iter("title"):
    if title.text != None:
        # print(title.text)

        text = title.text
        # find words contained inside parentheses and add to a variable
        if ("(" in text) and (")" in text):
            text = re.findall(r"\((.*?)\)", text)
            name = text[0]

        else:
            print("no name")

        break
    else:
        print("no title")
zip = f"/media/muskrat/T7 Shield/eco_data/v3/native/GAP/unzipped/{i}/{file_list[0]}.zip"
with ZipFile(zip, "r") as zObject:
    # print(zObject.namelist())
    # open file in zObject that ends in .csv and save to dataframe
    for j in zObject.namelist():
        if j.endswith(".csv"):
            # print(j)
            metadata = pd.read_csv(zObject.open(j))
            metadata["join"] = i
            meta_df = pd.concat([meta_df, metadata])
            break

# print(name)

name_df = pd.concat(
    [name_df, pd.DataFrame({"scientific_name": [name], "join": [i]})]
)

geodata = gpd.read_file(zip)
geodata["join"] = i
# drop season code column and season name column from df

geodata = geodata.drop(columns=["SeasonCode", "SeasonName"])

geo_df = pd.concat([geo_df, geodata])


# merge meta_df, name_df, and geo_df on join column

df = pd.merge(meta_df, name_df, on="join")

4


In [7]:
df = pd.merge(df, geo_df, on="join")

In [8]:
# drop column strHUC12RNG

df = df.drop(columns="strHUC12RNG")

In [9]:
df = gpd.GeoDataFrame(df, geometry="geometry")

In [10]:
del geo_df, name_df, meta_df, metadata, geodata

In [17]:
# drop duplicate rows

df = df.drop_duplicates()

In [ ]:
# convert to geodataframe


df.to_parquet("/media/muskrat/T7 Shield/eco_data/v3/native/GAP/init/0.parquet")

In [11]:
df

,strUC,intGapOrigin,intGapPres,intGapRepro,intGapSeas,Origin,Presence,Reproduction,Season,join,scientific_name,geometry
0,aAMBUx,1,1,3,1,Native,Known/extant,Both breeding and nonbreeding,Year-round,4,Lithobates catesbeianus,"MULTIPOLYGON (((-814736.750 2178393.250, -8147..."
1,aAMBUx,1,1,3,1,Native,Known/extant,Both breeding and nonbreeding,Year-round,4,Lithobates catesbeianus,"MULTIPOLYGON (((-1241661.875 2130988.750, -124..."
2,aAMBUx,1,1,3,1,Native,Known/extant,Both breeding and nonbreeding,Year-round,4,Lithobates catesbeianus,"MULTIPOLYGON (((-814736.750 2178393.250, -8147..."
3,aAMBUx,1,1,3,1,Native,Known/extant,Both breeding and nonbreeding,Year-round,4,Lithobates catesbeianus,"MULTIPOLYGON (((-1241661.875 2130988.750, -124..."
4,aAMBUx,1,1,3,1,Native,Known/extant,Both breeding and nonbreeding,Year-round,4,Lithobates catesbeianus,"MULTIPOLYGON (((-814736.750 2178393.250, -8147..."
...,...,...,...,...,...,...,...,...,...,...,...,...
129751,aAMBUx,2,1,3,1,Introduced,Known/extant,Both breeding and nonbreeding,Year-round,4,Lithobates catesbeianus,"MULTIPOLYGON (((-1241661.875 2130988.750, -124..."
129752,aAMBUx,2,1,3,1,Introduced,Known/extant,Both breeding and nonbreeding,Year-round,4,Lithobates catesbeianus,"MULTIPOLYGON (((-814736.750 2178393.250, -8147..."
129753,aAMBUx,2,1,3,1,Introduced,Known/extant,Both breeding and nonbreeding,Year-round,4,Lithobates catesbeianus,"MULTIPOLYGON (((-1241661.875 2130988.750, -124..."
129754,aAMBUx,2,1,3,1,Introduced,Known/extant,Both breeding and nonbreeding,Year-round,4,Lithobates catesbeianus,"MULTIPOLYGON (((-814736.750 2178393.250, -8147..."


In [ ]:
# remove duplicate rows

df = df.drop_duplicates()

In [ ]:
df.plot()

In [ ]:
df.crs

# check units
# df.crs.axis_info[0].unit_name

In [ ]:
# converts crs to epsg:4326
df = df.to_crs("EPSG:4326")
df.crs

In [ ]:
df.plot()

In [ ]:
ecomap_loc = "/media/muskrat/T7 Shield/eco_data/ecomap_final/eco_map.geojson"

eco_map = gpd.read_file(ecomap_loc)

In [ ]:
eco_map

In [ ]:
eco_map.crs

In [ ]:
# convert df to geodataframe maybe not needed
df = gpd.GeoDataFrame(df, geometry="geometry")

In [ ]:
# plot ecomap and df on same map
base = eco_map.plot(color="white", edgecolor="black")
xmin, ymin, xmax, ymax = (-120, 30, -100, 45)

ax = df.plot(ax=base, color="red", alpha=0.4)

# set the x and y limits of the plot to the specified bounding box coordinates
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
# plot the GeoDataFrame with the specified bounding box
# df.plot(ax=base, color='red', extent=[xmin, xmax, ymin, ymax])

# df.plot(ax=base, color='red')

In [ ]:
df

In [ ]:
# check if df geometry intersects with ecomap geometry

intersects = gpd.sjoin(df, eco_map)

In [ ]:
intersects.head()

In [ ]:
# put unique values of unique_id in intersects into a list

unique_ids = list(intersects["unique_id"].unique())
unique_ids

In [ ]:
# create dataframe from eco_map that only contains the unique ids in unique_ids

eco_map_unique = eco_map[eco_map["unique_id"].isin(unique_ids)]
# eco_map_unique

In [ ]:
eco_map_unique.head()

In [ ]:
overlay = gpd.overlay(df, eco_map, how="intersection")

In [ ]:
overlay.plot(alpha=0.5, edgecolor="k", cmap="tab10")

In [ ]:
overlay

In [ ]:
o_area = overlay.area.sum()

o_area

In [ ]:
# add area column to overlay dataframe

overlay["area"] = overlay.geometry.area

In [ ]:
# find row with max area in overlay dataframe

overlay.loc[overlay["area"].idxmax()]

In [ ]:
# print overlay row

overlay.iloc[6]

In [ ]:
# create a new dataframe from overlay where the first column is unique_id and the second column is the area of all the rows in overlay that have the same unique_id

overlay_areas = overlay[["unique_id", "area"]].groupby("unique_id").sum()

In [ ]:
overlay_areas

In [ ]:
# add an area column to eco_map_unique dataframe

eco_map_unique["area"] = eco_map_unique.geometry.area

In [ ]:
# create a new dataframe from eco_map_unique where the first column is unique_id and the second column is the area of all the rows in eco_map_unique that have the same unique_id

eco_map_unique_areas = eco_map_unique[["unique_id", "area"]].groupby("unique_id").sum()

In [ ]:
eco_map_unique_areas

In [ ]:
# combine eco_map_unique_areas and overlay_areas into a new dataframe where the first column is unique_id, the second column is area from overlays, and the third column is area from eco_map_unique

combined_areas = pd.concat([overlay_areas, eco_map_unique_areas], axis=1)
combined_areas.columns = ["overlay_area", "eco_map_unique_area"]

combined_areas

In [ ]:
# if overlay_area / eco_map_unique_area > 0.2 then add unique_id to list of ids

native = combined_areas[
    combined_areas["overlay_area"] / combined_areas["eco_map_unique_area"] > 0.2
].index.tolist()

native

In [ ]:
# inter_repro = intersects.to_crs("EPSG:6933")

# inter_repro.crs.axis_info[0].unit_name